In [99]:
import os


In [100]:
%pwd

'C:\\Users'

In [101]:
os.chdir("../")

In [102]:
%pwd

'C:\\'

In [103]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [104]:
from src.InfoCondense.constants import *
from src.InfoCondense.utils.common import read_yaml, create_directories



In [114]:
class ConfigManager:
    def __init__(
            self, 
            config_path: CONFIG_FILE_PATH,
            params_path: PARAMS_FILE_PATH):
            self.config = read_yaml(config_path)
            self.params = read_yaml(params_path) 
            create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url, 
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [115]:
import os
import urllib.request as request
import zipfile
from src.InfoCondense.logging import logger
from src.InfoCondense.utils.common import get_size



In [122]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n {headers}")
            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    def extract_zip(self):
        """
        zip_file_path:str
        Extract zip file from url into the data directory
        Function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
             

In [124]:
try: 
    config = ConfigManager(CONFIG_FILE_PATH,PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2024-07-10 14:41:25,855: INFO: common]: yaml file: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\config\config.yaml loaded successfully]
[2024-07-10 14:41:25,863: INFO: common]: yaml file: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\params.yaml loaded successfully]
[2024-07-10 14:41:25,863: INFO: common]: created directory at: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\artifacts]
[2024-07-10 14:41:25,863: INFO: common]: created directory at: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\artifacts\data_ingestion]
[2024-07-10 14:41:40,541: INFO: 4286236413]: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\artifacts\data_ingestion\data.zip download! with the following info: 
 Connection: close
Content-Length: 8398633
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "42e